In [1]:
import numpy as np
import h5py

import e2c as e2c_util

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1", "2", "3"

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras import losses


# GPU memory management
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# tf.session specification
# TensorFlow wizardry
config = tf.ConfigProto()

# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.75

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
def reconstruction_loss(x, t_decoded):
    '''
    Reconstruction loss for the plain VAE
    '''
    v = 0.1
    # return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v) + 0.5*K.log(2*np.pi*v), axis=-1))
    return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v), axis=-1))
    # return K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2, axis=-1)


def l2_reg_loss(qm):
    # 0.5 * (torch.log(pv) - torch.log(qv) + qv / pv + (qm - pm).pow(2) / pv - 1)
    # -0.5 * K.sum(1 + t_log_var - K.square(t_mean) - K.exp(t_log_var), axis=-1)
#     kl = -0.5 * (1 - p_logv + q_logv - K.exp(q_logv) / K.exp(p_logv) - K.square(qm - pm) / K.exp(p_logv))
    l2_reg = 0.5*K.square(qm)
    return K.mean(K.sum(l2_reg, axis=-1))


def get_flux_loss(m, state, state_pred):
    '''
    @params:  state, state_pred shape (batch_size, 60, 60, 2)
              p, p_pred shape (batch_size, 60, 60, 1)
              m shape (batch_size, 60, 60, 1)
    
    @return:  loss_flux: scalar
    
    Only consider discrepancies in total flux, not in phases (saturation not used) 
    '''
    
    perm = K.exp(m)
    p = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)

    #print(K.int_shape(xxx))
    
    tran_x = 1./perm[:, 1:, ...] + 1./perm[:, :-1, ...]
    tran_y = 1./perm[:, :, 1:, ...] + 1./perm[:, :, :-1, ...]
    flux_x = (p[:, 1:, ...] - p[:, :-1, ...]) / tran_x
    flux_y = (p[:, :, 1:, :] - p[:, :, :-1, :]) / tran_y
    flux_x_pred = (p_pred[:, 1:, ...] - p_pred[:, :-1, ...]) / tran_x
    flux_y_pred = (p_pred[:, :, 1:, :] - p_pred[:, :, :-1, :]) / tran_y

    loss_x = K.sum(K.abs(K.batch_flatten(flux_x) - K.batch_flatten(flux_x_pred)), axis=-1)
    loss_y = K.sum(K.abs(K.batch_flatten(flux_y) - K.batch_flatten(flux_y_pred)), axis=-1)

    loss_flux = K.mean(loss_x + loss_y)
    return loss_flux

In [4]:
def get_binary_sat_loss(state, state_pred):
    
    sat_threshold = 0.105
    sat = K.expand_dims(state[:, :, :, 0], -1)
    sat_pred = K.expand_dims(state_pred[:, :, :, 0], -1)
    
    
    sat_bool = K.greater_equal(sat, sat_threshold) #will return boolean values
    sat_bin = K.cast(sat_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
    sat_pred_bool = K.greater_equal(sat_pred, sat_threshold) #will return boolean values
    sat_pred_bin = K.cast(sat_pred_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
#     binary_loss = K.sum(K.abs(K.batch_flatten(sat_bin) - K.batch_flatten(sat_pred_bin)), axis=-1)
    
    binary_loss = losses.binary_crossentropy(sat_bin, sat_pred_bin)
    return K.mean(binary_loss)

In [5]:
def get_well_bhp_loss(state, state_pred, prod_well_loc):
    '''
    @params: state: shape (batch_size, 60, 60, 2)
             state_pred: shape (batch_size, 60, 60, 2)
             prod_well_loc: shape (5, 2)
             
    p_true: shape (batch_size, 60, 60, 1)
    p_pred: shape (batch_size, 60, 60, 1)
    
    @return: bhp_loss: scalar
    '''
    
    p_true = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)
    
    print(K.int_shape(prod_well_loc))
#     print(K.int_shape(prod_well_loc[0, :, 0]))
#     print(K.int_shape(prod_well_loc[0, :, 1]))
    print(K.int_shape(prod_well_loc[:, 0]))
    print(K.int_shape(prod_well_loc[:, 1]))
    print(prod_well_loc.shape[0])
    bhp_loss = 0
    for i in range(prod_well_loc.shape[0]):
        bhp_loss += K.mean(K.abs(p_true[:, prod_well_loc[i, 1], prod_well_loc[i, 0], :] - p_pred[:, prod_well_loc[i, 1], prod_well_loc[i, 0], :]))
#     bhp_loss = K.sum(p_true[:, prod_well_loc[:, 0], prod_well_loc[:, 1], :] - p_pred[:, prod_well_loc[:,0], prod_well_loc[:,1], :])
    
    return bhp_loss

In [6]:
def create_e2c(latent_dim, u_dim, input_shape, sigma=0.0):
    '''
    Creates a E2C.

    Args:
        latent_dim: dimensionality of latent space
        return_kl_loss_op: whether to return the operation for
                           computing the KL divergence loss.

    Returns:
        The VAE model. If return_kl_loss_op is True, then the
        operation for computing the KL divergence loss is
        additionally returned.
    '''

    encoder_ = e2c_util.create_encoder(latent_dim, input_shape, sigma=sigma)
    decoder_ = e2c_util.create_decoder(latent_dim, input_shape)
    transition_ = e2c_util.create_trans(latent_dim, u_dim)

    return encoder_, decoder_, transition_

In [7]:
# Create plain E2C model and associated loss operations

################### case specification ######################

# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP_RATE/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_MS_BHP_RATE/'
# data_dir = '/data3/Astro/personal/zjin/datasets/9W_MS_BHP_RATE_GAU/'
data_dir = '/data3/Astro/personal/zjin/datasets/7W_CHA/'

output_dir = '/data3/Astro/lstm_rom/e2c_larry/saved_models/'

# case_name = '9w_bhp'
# case_name = '9w_bhp_rate'
# case_name = '9w_ms_bhp_rate'
case_name = '7w_cha'

# case_suffix = '_single_out_rel_2'
case_suffix = '_fix_wl_rel_1'

train_suffix = '_with_p'

# model_suffix = '_flux_loss'
model_suffix = '_ae_no_l2_ep_10'


n_train_run = 300
n_eval_run = 100
num_t = 20 
# dt = 200 // num_t
dt = 100
n_train_step = n_train_run * num_t
n_eval_step = n_eval_run * num_t


train_file = case_name + '_e2c_train' + case_suffix + train_suffix + '_n%d_dt%dday_nt%d_nrun%d.mat' %(n_train_step, dt, num_t, n_train_run)
eval_file = case_name + '_e2c_eval' + case_suffix + train_suffix +'_n%d_dt%dday_nt%d_nrun%d.mat' %(n_eval_step, dt, num_t, n_eval_run)

#################### model specification ##################
epoch = 10
batch_size = 4
learning_rate = 1e-4
latent_dim = 50

u_dim = 7*2 # control dimension

# load data
hf_r = h5py.File(data_dir + train_file, 'r')
state_t_train = np.array(hf_r.get('state_t'))
state_t1_train = np.array(hf_r.get('state_t1'))
bhp_train = np.array(hf_r.get('bhp'))
dt_train = np.array(hf_r.get('dt'))
hf_r.close()

num_train = state_t_train.shape[0]
# dt_train = np.ones((num_train,1)) # dt=20days, normalized to 1

hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
state_t1_eval = np.array(hf_r.get('state_t1'))
bhp_eval = np.array(hf_r.get('bhp'))
dt_eval = np.array(hf_r.get('dt'))
hf_r.close()

num_eval = state_t_eval.shape[0]
# dt_eval = np.ones((num_eval, 1)) # dt=20days, normalized to 1

In [8]:
# m = np.loadtxt("/data/cees/zjin/lstm_rom/sim_runs/case6_9w_bhp_rate_ms_h5/template/logk1.dat")
# m = np.loadtxt("/data3/Astro/personal/zjin/sim_runs/case8_9w_bhp_rate_ms_gau/template/logk1.dat") # Gaussian
m = np.loadtxt("/data3/Astro/personal/zjin/sim_runs/case9_cha/template/logk1.dat") # channelized

m = m.reshape(60, 60, 1)
print('m shape is ', m.shape)
#     m_tf = K.placeholder((batch_size, 60, 60 ,1))
m_tf = Input(shape=(60, 60, 1))


m_eval = np.repeat(np.expand_dims(m, axis = 0), state_t_eval.shape[0], axis = 0)
print("m_eval shape is ", m_eval.shape)
m = np.repeat(np.expand_dims(m,axis = 0), state_t_train.shape[0], axis = 0)
print("m shape is ", m.shape)

m shape is  (60, 60, 1)
m_eval shape is  (2000, 60, 60, 1)
m shape is  (6000, 60, 60, 1)


In [9]:
# well_loc_file = '/data3/Astro/personal/zjin/sim_runs/case8_9w_bhp_rate_ms_gau/template/well_loc00.dat'
well_loc_file = '/data3/Astro/personal/zjin/sim_runs/case9_cha/template/well_loc00.dat'
well_loc = np.loadtxt(well_loc_file).astype(int)
num_prod = well_loc[0,0]
num_inj = well_loc[0,1]
num_well = num_prod+num_inj
print(num_inj, num_prod)

prod_loc = well_loc[1:num_prod+1,:]
print("prod_loc:\n{}".format(prod_loc))
# inj_loc = well_loc[num_prod+1:,:]
# print("inj_loc:\n{}".format(inj_loc))

print(prod_loc.shape)

print('prod_loc shape is ', prod_loc.shape)
# prod_loc_tf = Input(shape=(5, 2), dtype ='int32')
prod_loc_tf = tf.placeholder(tf.int32, shape=(num_prod,2))

# prod_loc_eval = np.repeat(np.expand_dims(prod_loc, axis = 0), state_t_eval.shape[0], axis = 0)
# print("prod_loc_eval shape is ", prod_loc_eval.shape)
# prod_loc = np.repeat(np.expand_dims(prod_loc,axis = 0), state_t_train.shape[0], axis = 0)
# print("prod_loc shape is ", prod_loc.shape)

3 4
prod_loc:
[[11 10]
 [14 50]
 [40 30]
 [55 20]]
(4, 2)
prod_loc shape is  (4, 2)


In [10]:
# import importlib
# importlib.reload(e2c_util)


In [11]:
# Construct E2C
input_shape = (60, 60, 2)

#############################################
# here we use a UAE framework, sigma = 0.0001
#############################################
encoder, decoder, transition = create_e2c(latent_dim, u_dim, input_shape, sigma=0.0) 


xt = Input(shape=input_shape)
xt1 = Input(shape=input_shape)
ut = Input(shape=(u_dim, ))
dt = Input(shape=(1,))

zt = encoder(xt)
xt_rec = decoder(zt)

zt1 = encoder(xt1)

zt1_pred = transition([zt, ut, dt])
xt1_pred = decoder(zt1_pred)

# Compute loss
loss_rec_t = reconstruction_loss(xt, xt_rec)
loss_rec_t1 = reconstruction_loss(xt1, xt1_pred)

loss_flux_t = get_flux_loss(m_tf, xt, xt_rec) / 1000.
loss_flux_t1 = get_flux_loss(m_tf, xt1, xt1_pred) / 1000.

binary_sat_loss_t = get_binary_sat_loss(xt, xt_rec) * 1
binary_sat_loss_t1 = get_binary_sat_loss(xt1, xt1_pred) * 1

loss_prod_bhp_t = get_well_bhp_loss(xt, xt_rec, prod_loc_tf) * 20
loss_prod_bhp_t1 = get_well_bhp_loss(xt1, xt1_pred, prod_loc_tf) * 20

loss_l2_reg = l2_reg_loss(zt)  # log(1.) = 0.
# loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1
# loss_bound = loss_rec_t + loss_rec_t1 + loss_kl + binary_sat_loss_t + binary_sat_loss_t1
# loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1 + loss_prod_bhp_t + loss_prod_bhp_t1 # JCP 2019 Gaussian case
loss_bound = loss_rec_t + loss_rec_t1  + loss_flux_t + loss_flux_t1 + loss_prod_bhp_t + loss_prod_bhp_t1 # UAE




# Use zt_logvar to approximate zt1_logvar_pred
loss_trans = l2_reg_loss(zt1_pred - zt1)
# loss_trans = kl_normal_loss(zt1_mean_pred, zt1_logvar_pred, zt1_mean, zt1_logvar)


trans_loss_weight = 1.0 # lambda in E2C paper Eq. (11)
loss = loss_bound + trans_loss_weight * loss_trans

(4, 2)
(4,)
(4,)
4
(4, 2)
(4,)
(4,)
4


In [12]:
# Optimization
opt = Adam(lr=learning_rate)

trainable_weights = encoder.trainable_weights + decoder.trainable_weights + transition.trainable_weights

updates = opt.get_updates(loss, trainable_weights)

# iterate = K.function([xt, ut, xt1, m_tf, dt], [loss, loss_rec_t, loss_rec_t1, loss_l2_reg, loss_trans, loss_flux_t, loss_flux_t1, binary_sat_loss_t, binary_sat_loss_t1], updates=updates)
iterate = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss, loss_rec_t, loss_rec_t1, loss_l2_reg, loss_trans, loss_flux_t, loss_flux_t1, loss_prod_bhp_t, loss_prod_bhp_t1], updates=updates)

eval_loss = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss])

num_batch = int(num_train/batch_size)

for e in range(epoch):
    for ib in range(num_batch):
        ind0 = ib * batch_size
        state_t_batch  = state_t_train[ind0:ind0+batch_size, ...]
        state_t1_batch = state_t1_train[ind0:ind0 + batch_size, ...]
        bhp_batch      = bhp_train[ind0:ind0 + batch_size, ...]
        m_batch        = m[ind0:ind0 + batch_size, ...]
        dt_batch       = dt_train[ind0:ind0 + batch_size, ...]
        
#         print(state_t_batch.shape)
#         print(state_t1_batch.shape)
#         print(bhp_batch.shape)
#         print(m_batch.shape)
#         print(dt_batch.shape)
        

        output = iterate([state_t_batch, bhp_batch, state_t1_batch, m_batch, prod_loc, dt_batch])

        # tf.session.run(feed_dict={xt: sat_t_batch, ut: bhp_batch, xt1: sat_t1_batch}, ...
        #                fetches= [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, updates])
        # But output tensor for the updates operation is not returned

        if ib % 10 == 0:
            print('Epoch %d/%d, Batch %d/%d, Loss %f, Loss rec %f, loss rec t1 %f, loss kl %f, loss_trans %f, loss flux %f, loss flux t1 %f, prod bhp loss %f, prod bhp loss t1 %f'
                  % (e+1, epoch, ib+1, num_batch, output[0], output[1], output[2], output[3], output[4], output[5], output[6], output[7], output[8]))
    eval_loss_val = eval_loss([state_t_eval, bhp_eval, state_t1_eval, m_eval, prod_loc, dt_eval])
    
    print('====================================================')
    print('\n')
    print('Epoch %d/%d, Train loss %f, Eval loss %f' % (e + 1, epoch, output[0], eval_loss_val[0]))
    print('\n')
    print('====================================================')

encoder.save_weights(output_dir + 'e2c_encoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
decoder.save_weights(output_dir + 'e2c_decoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
transition.save_weights(output_dir + 'e2c_transition_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                        % (num_train, latent_dim, learning_rate, epoch))

Epoch 1/10, Batch 1/1500, Loss 5558.417480, Loss rec 2697.260986, loss rec t1 2806.703125, loss kl 0.023967, loss_trans 0.033807, loss flux 3.190595, loss flux t1 2.607153, prod bhp loss 24.116030, prod bhp loss t1 24.506310
Epoch 1/10, Batch 11/1500, Loss 1117.122437, Loss rec 556.560181, loss rec t1 402.366272, loss kl 18.783484, loss_trans 22.047966, loss flux 75.249313, loss flux t1 48.549503, prod bhp loss 9.404984, prod bhp loss t1 2.944132
Epoch 1/10, Batch 21/1500, Loss 702.377380, Loss rec 289.098755, loss rec t1 324.427338, loss kl 8.730597, loss_trans 12.809248, loss flux 35.098499, loss flux t1 34.834282, prod bhp loss 2.712328, prod bhp loss t1 3.396869
Epoch 1/10, Batch 31/1500, Loss 556.547852, Loss rec 258.389679, loss rec t1 209.250885, loss kl 8.266649, loss_trans 9.681475, loss flux 38.675964, loss flux t1 36.225712, prod bhp loss 2.555230, prod bhp loss t1 1.768916
Epoch 1/10, Batch 41/1500, Loss 367.595398, Loss rec 135.491333, loss rec t1 151.664505, loss kl 6.976

Epoch 1/10, Batch 371/1500, Loss 117.810669, Loss rec 46.783051, loss rec t1 50.766308, loss kl 4.861379, loss_trans 0.940720, loss flux 8.484818, loss flux t1 8.084676, prod bhp loss 1.562327, prod bhp loss t1 1.188770
Epoch 1/10, Batch 381/1500, Loss 111.308617, Loss rec 46.594162, loss rec t1 47.271503, loss kl 4.170383, loss_trans 0.594440, loss flux 6.607953, loss flux t1 6.998712, prod bhp loss 1.561453, prod bhp loss t1 1.680395
Epoch 1/10, Batch 391/1500, Loss 95.696365, Loss rec 38.963837, loss rec t1 34.993687, loss kl 4.470302, loss_trans 0.586028, loss flux 9.662950, loss flux t1 8.805418, prod bhp loss 1.535365, prod bhp loss t1 1.149085
Epoch 1/10, Batch 401/1500, Loss 122.751434, Loss rec 44.880119, loss rec t1 54.444481, loss kl 4.422731, loss_trans 0.687360, loss flux 10.287032, loss flux t1 8.579393, prod bhp loss 1.824841, prod bhp loss t1 2.048201
Epoch 1/10, Batch 411/1500, Loss 83.491898, Loss rec 36.418816, loss rec t1 32.122330, loss kl 4.567758, loss_trans 0.63

Epoch 1/10, Batch 751/1500, Loss 56.931309, Loss rec 21.502426, loss rec t1 22.338118, loss kl 4.240947, loss_trans 0.148608, loss flux 5.050872, loss flux t1 5.777103, prod bhp loss 1.192232, prod bhp loss t1 0.921948
Epoch 1/10, Batch 761/1500, Loss 46.073013, Loss rec 14.376986, loss rec t1 18.947830, loss kl 3.918057, loss_trans 0.157278, loss flux 5.917460, loss flux t1 5.498823, prod bhp loss 0.404280, prod bhp loss t1 0.770355
Epoch 1/10, Batch 771/1500, Loss 54.854286, Loss rec 20.300089, loss rec t1 21.947170, loss kl 4.133879, loss_trans 0.131699, loss flux 5.434593, loss flux t1 5.463981, prod bhp loss 0.863470, prod bhp loss t1 0.713283
Epoch 1/10, Batch 781/1500, Loss 52.698326, Loss rec 15.159437, loss rec t1 25.292706, loss kl 3.533961, loss_trans 0.158271, loss flux 4.951517, loss flux t1 5.284524, prod bhp loss 0.417255, prod bhp loss t1 1.434622
Epoch 1/10, Batch 791/1500, Loss 54.388977, Loss rec 20.679111, loss rec t1 21.619955, loss kl 3.823470, loss_trans 0.156743

Epoch 1/10, Batch 1131/1500, Loss 41.781017, Loss rec 14.957429, loss rec t1 17.280926, loss kl 3.905016, loss_trans 0.044171, loss flux 4.278260, loss flux t1 4.334513, prod bhp loss 0.311746, prod bhp loss t1 0.573975
Epoch 1/10, Batch 1141/1500, Loss 40.579697, Loss rec 15.260684, loss rec t1 16.215351, loss kl 3.050403, loss_trans 0.053349, loss flux 4.120320, loss flux t1 4.070097, prod bhp loss 0.368928, prod bhp loss t1 0.490966
Epoch 1/10, Batch 1151/1500, Loss 44.463657, Loss rec 16.528278, loss rec t1 17.045923, loss kl 3.686054, loss_trans 0.075313, loss flux 4.466864, loss flux t1 4.624785, prod bhp loss 0.803953, prod bhp loss t1 0.918535
Epoch 1/10, Batch 1161/1500, Loss 34.011841, Loss rec 10.627545, loss rec t1 13.581696, loss kl 3.060379, loss_trans 0.066503, loss flux 4.373745, loss flux t1 4.264360, prod bhp loss 0.248992, prod bhp loss t1 0.849002
Epoch 1/10, Batch 1171/1500, Loss 39.339718, Loss rec 13.196642, loss rec t1 15.951878, loss kl 3.360626, loss_trans 0.0

Epoch 2/10, Batch 11/1500, Loss 26.898586, Loss rec 9.137373, loss rec t1 9.837282, loss kl 3.476586, loss_trans 0.045584, loss flux 3.561496, loss flux t1 3.539397, prod bhp loss 0.392141, prod bhp loss t1 0.385311
Epoch 2/10, Batch 21/1500, Loss 28.787882, Loss rec 9.784053, loss rec t1 11.305168, loss kl 2.677118, loss_trans 0.049978, loss flux 3.290662, loss flux t1 3.360653, prod bhp loss 0.419471, prod bhp loss t1 0.577895
Epoch 2/10, Batch 31/1500, Loss 30.163481, Loss rec 10.104627, loss rec t1 11.777049, loss kl 3.447794, loss_trans 0.063041, loss flux 3.477600, loss flux t1 3.768762, prod bhp loss 0.428133, prod bhp loss t1 0.544270
Epoch 2/10, Batch 41/1500, Loss 24.527672, Loss rec 8.465075, loss rec t1 8.880523, loss kl 2.647216, loss_trans 0.025788, loss flux 3.216746, loss flux t1 3.375162, prod bhp loss 0.250933, prod bhp loss t1 0.313448
Epoch 2/10, Batch 51/1500, Loss 30.711172, Loss rec 11.182787, loss rec t1 11.778459, loss kl 2.807249, loss_trans 0.037457, loss flu

...

Epoch 9/10, Batch 1231/1500, Loss 7.373999, Loss rec 2.030317, loss rec t1 2.211358, loss kl 0.738198, loss_trans 0.003449, loss flux 1.309925, loss flux t1 1.274610, prod bhp loss 0.268996, prod bhp loss t1 0.275344
Epoch 9/10, Batch 1241/1500, Loss 7.962810, Loss rec 2.612573, loss rec t1 2.346924, loss kl 0.614061, loss_trans 0.004855, loss flux 1.169172, loss flux t1 1.128881, prod bhp loss 0.386035, prod bhp loss t1 0.314371
Epoch 9/10, Batch 1251/1500, Loss 7.781043, Loss rec 1.210757, loss rec t1 3.133915, loss kl 0.651084, loss_trans 0.005710, loss flux 1.271737, loss flux t1 1.170714, prod bhp loss 0.308670, prod bhp loss t1 0.679539
Epoch 9/10, Batch 1261/1500, Loss 7.666081, Loss rec 2.119154, loss rec t1 2.407726, loss kl 0.767534, loss_trans 0.003767, loss flux 1.244793, loss flux t1 1.283655, prod bhp loss 0.312905, prod bhp loss t1 0.294081
Epoch 9/10, Batch 1271/1500, Loss 8.390709, Loss rec 3.191940, loss rec t1 2.034900, loss kl 0.390491, loss_trans 0.007939, loss flu

Epoch 10/10, Batch 111/1500, Loss 6.581245, Loss rec 1.751268, loss rec t1 2.147576, loss kl 0.439293, loss_trans 0.004018, loss flux 1.042883, loss flux t1 1.058155, prod bhp loss 0.193819, prod bhp loss t1 0.383526
Epoch 10/10, Batch 121/1500, Loss 9.138255, Loss rec 2.626280, loss rec t1 3.247645, loss kl 0.660837, loss_trans 0.004479, loss flux 1.365694, loss flux t1 1.397465, prod bhp loss 0.209323, prod bhp loss t1 0.287370
Epoch 10/10, Batch 131/1500, Loss 6.834344, Loss rec 1.974254, loss rec t1 2.255687, loss kl 0.494804, loss_trans 0.002982, loss flux 0.983958, loss flux t1 0.968264, prod bhp loss 0.289619, prod bhp loss t1 0.359580
Epoch 10/10, Batch 141/1500, Loss 8.847207, Loss rec 2.756475, loss rec t1 3.054034, loss kl 0.772102, loss_trans 0.005961, loss flux 1.253192, loss flux t1 1.327652, prod bhp loss 0.175199, prod bhp loss t1 0.274694
Epoch 10/10, Batch 151/1500, Loss 6.048834, Loss rec 1.408659, loss rec t1 1.905105, loss kl 0.738323, loss_trans 0.003916, loss flu

Epoch 10/10, Batch 491/1500, Loss 6.208865, Loss rec 1.978496, loss rec t1 2.010660, loss kl 0.553078, loss_trans 0.003894, loss flux 0.996451, loss flux t1 1.008543, prod bhp loss 0.102902, prod bhp loss t1 0.107919
Epoch 10/10, Batch 501/1500, Loss 6.671218, Loss rec 1.875449, loss rec t1 2.087905, loss kl 0.739610, loss_trans 0.002462, loss flux 1.129276, loss flux t1 1.129392, prod bhp loss 0.186004, prod bhp loss t1 0.260729
Epoch 10/10, Batch 511/1500, Loss 7.798089, Loss rec 2.592031, loss rec t1 1.997621, loss kl 0.619527, loss_trans 0.004221, loss flux 1.195989, loss flux t1 1.172771, prod bhp loss 0.524996, prod bhp loss t1 0.310460
Epoch 10/10, Batch 521/1500, Loss 8.247797, Loss rec 2.755575, loss rec t1 2.301843, loss kl 0.564551, loss_trans 0.004482, loss flux 1.135314, loss flux t1 1.222000, prod bhp loss 0.476372, prod bhp loss t1 0.352210
Epoch 10/10, Batch 531/1500, Loss 10.013845, Loss rec 4.009333, loss rec t1 2.400103, loss kl 0.593534, loss_trans 0.004981, loss fl

Epoch 10/10, Batch 871/1500, Loss 6.469492, Loss rec 1.770235, loss rec t1 1.996454, loss kl 0.360748, loss_trans 0.006019, loss flux 1.044570, loss flux t1 1.110501, prod bhp loss 0.271781, prod bhp loss t1 0.269931
Epoch 10/10, Batch 881/1500, Loss 5.967462, Loss rec 1.720734, loss rec t1 1.997723, loss kl 0.394301, loss_trans 0.003860, loss flux 0.973303, loss flux t1 0.978851, prod bhp loss 0.132066, prod bhp loss t1 0.160924
Epoch 10/10, Batch 891/1500, Loss 8.496265, Loss rec 3.017940, loss rec t1 2.199183, loss kl 0.502837, loss_trans 0.004070, loss flux 1.140146, loss flux t1 1.175000, prod bhp loss 0.595883, prod bhp loss t1 0.364044
Epoch 10/10, Batch 901/1500, Loss 7.376000, Loss rec 1.929585, loss rec t1 2.351067, loss kl 0.553931, loss_trans 0.003613, loss flux 1.255625, loss flux t1 1.195320, prod bhp loss 0.263091, prod bhp loss t1 0.377701
Epoch 10/10, Batch 911/1500, Loss 5.666412, Loss rec 1.692766, loss rec t1 1.605367, loss kl 0.571687, loss_trans 0.003011, loss flu

Epoch 10/10, Batch 1251/1500, Loss 6.440379, Loss rec 1.852501, loss rec t1 1.668891, loss kl 0.616991, loss_trans 0.005898, loss flux 1.112898, loss flux t1 0.990721, prod bhp loss 0.505983, prod bhp loss t1 0.303487
Epoch 10/10, Batch 1261/1500, Loss 7.037385, Loss rec 2.091553, loss rec t1 2.229896, loss kl 0.696199, loss_trans 0.003613, loss flux 1.119685, loss flux t1 1.135739, prod bhp loss 0.245591, prod bhp loss t1 0.211309
Epoch 10/10, Batch 1271/1500, Loss 7.766445, Loss rec 2.896286, loss rec t1 1.815588, loss kl 0.354923, loss_trans 0.007813, loss flux 1.061389, loss flux t1 1.112938, prod bhp loss 0.612957, prod bhp loss t1 0.259474
Epoch 10/10, Batch 1281/1500, Loss 7.542306, Loss rec 2.142096, loss rec t1 2.319616, loss kl 0.537999, loss_trans 0.005890, loss flux 1.228824, loss flux t1 1.218373, prod bhp loss 0.312853, prod bhp loss t1 0.314654
Epoch 10/10, Batch 1291/1500, Loss 8.804303, Loss rec 2.095955, loss rec t1 3.293757, loss kl 0.572401, loss_trans 0.005543, los